# Przykład do metody ALS

### Disclaimer  
Nie zwracajcie uwagi na kod tylko na sam przykład aby zobaczyć jak to się liczy   

---

Tu nie jest nic ważnego, importowanie paczek itd.

In [30]:
import pandas as pd
import numpy as np

gauss_method = np.linalg.solve

Mamy przykładowe dane:

In [31]:
example_data = pd.read_csv('example_data.csv', sep=';')

example_data

,id_user,id_product,rating,category
0,0,4,4,Books
1,0,6,5,Books
2,0,7,4,Books
3,1,0,4,Books
4,1,2,4,Books
5,1,5,4,Books
6,1,9,4,Books
7,2,0,5,Books
8,2,1,4,Books
9,2,2,5,Books


Z podanych danych tworzymy tabele przestawną (_pivot table_) i dla użytkowników którzy nie ocenili produktu uzupełniamy zerami

In [32]:
ratings = pd.pivot_table(example_data, values=['rating'], columns=['id_product'], index=['id_user'],
                                 aggfunc='first', fill_value=0)
ratings['rating']

id_product,0,1,2,3,4,5,6,7,8,9
id_user,,,,,,,,,,
0,0,0,0,0,4,0,5,4,0,0
1,4,0,4,0,0,4,0,0,0,4
2,5,4,5,5,0,5,5,5,5,5
3,0,5,5,0,5,0,0,5,0,5
4,0,5,5,0,5,0,0,5,0,5


Tak wygląda nasza docelowa macierz.  
Każdy **wiersz** reprezenuje jakiegoś użytkownika który dał oceny  
Każda **kolumna** reprezentuje jakiś produkt który posiada jakieś oceny

In [33]:
ratings_matrix = ratings.values
ratings_matrix

array([[0, 0, 0, 0, 4, 0, 5, 4, 0, 0],
       [4, 0, 4, 0, 0, 4, 0, 0, 0, 4],
       [5, 4, 5, 5, 0, 5, 5, 5, 5, 5],
       [0, 5, 5, 0, 5, 0, 0, 5, 0, 5],
       [0, 5, 5, 0, 5, 0, 0, 5, 0, 5]])

Teraz jak już mamy przygotowane dane możemy zacząć implementować algorytm ALS  
**Patrz zad3.pdf z portalu edukacyjnego, strona 2**

Najpierw przyjmimy pewne wartości które sami ustalamy np.:  
d = 3   
&lambda; = 0.1

In [34]:
d = 3
reg = 0.1 # lambda

#### 1. Nadaj wartości początkowe macierzom **P** i **U**  
Macierz P ma wymiary 3×10 (d×'ilość produktów')  
Macierz U ma wymiary 3×5  (d×'ilość użytkowników')  
Macierze uzupełniamy losowymi wartościami z zakresu _(0,1)_

Macierz P

In [35]:
P = np.random.rand(d, 10)
P

array([[0.93119636, 0.01215318, 0.82254304, 0.92704314, 0.72097256,
        0.1119594 , 0.05907673, 0.27337659, 0.51578453, 0.47299487],
       [0.1671686 , 0.02328032, 0.64793332, 0.46310597, 0.98508579,
        0.23390272, 0.34862754, 0.29751156, 0.81994987, 0.32293732],
       [0.72302848, 0.91165485, 0.70980305, 0.20125138, 0.33071352,
        0.40941998, 0.6984816 , 0.94986196, 0.52719633, 0.66722182]])

Macierz U

In [36]:
U = np.random.rand(d, 3)
U

array([[0.02930222, 0.90635812, 0.71271017],
       [0.03319273, 0.2316068 , 0.96492267],
       [0.35638381, 0.42064508, 0.83929454]])

#### 2. Powtarzaj do osiągnięcia zadowalającego rezultatu:  
Tutaj musicie sami zdecydować jaki to jest zadawaljący rezultat np. 100 albo 300 iteracji kroku 3,4,5,6

#### 3. for u = 1, 2, . . . n
n = ilość użytkowników  
Tutaj zrobimy dla u = 0, (dla kolejnych _u_ analogicznie)

In [37]:
u = 0

Najpierw potrzebujemy _I\_u_, czyli indeksy produktów którzy otrzymali ocenę (w naszym przypadku różną od 0) od użytkownika _u_  
Bierzemy wiersz o indeksie u=0 i sprawdzamy gdzie ocena jest rózna od 0

In [38]:
I_u = np.flatnonzero(ratings_matrix[u])
I_u

array([4, 6, 7])

Teraz możemy policzyć _P\_I\_u_ (czyli kolumny z macierzy _P_ o indeksach w _I\_u_)

In [39]:
P_I_u = P[:, I_u]
P_I_u

array([[0.72097256, 0.05907673, 0.27337659],
       [0.98508579, 0.34862754, 0.29751156],
       [0.33071352, 0.6984816 , 0.94986196]])

Przy okazji możemy policzyć transponowane _P\_I_u_

In [40]:
P_I_u_T = P_I_u.T
P_I_u_T

array([[0.72097256, 0.98508579, 0.33071352],
       [0.05907673, 0.34862754, 0.6984816 ],
       [0.27337659, 0.29751156, 0.94986196]])

Zostaje nam do policzenia macierz _E_ która jest macierzą jednostkową 3×3 (d×d)

In [41]:
E = np.eye(d)
E

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

Teraz możemy policzyć macierz _A\_u_

In [42]:
A_u = np.matmul(P_I_u, P_I_u_T) + reg * E
A_u

array([[0.69802625, 0.81214829, 0.5393694 ],
       [0.81214829, 1.2804483 , 0.85188602],
       [0.5393694 , 0.85188602, 1.59948571]])

Następnie liczymy _V\_u_, czyli musimy dodać do siebie:
 * kolumnę 4 z macierzy _P_ pomnożoną przez 4 (ocena)
 * kolumnę 6 z macierzy _P_ pomnożoną przez 5 (ocena)
 * kolumnę 7 z macierzy _P_ pomnożoną przez 4 (ocena)
 
 
 To wygląda mniej więcej tak:

In [43]:
for i in I_u:
    print('rating: {}, P_{} '.format(ratings_matrix[u][i], i))
    print(P[:, i].reshape(3,1))
    print()

rating: 4, P_4 
[[0.72097256]
 [0.98508579]
 [0.33071352]]

rating: 5, P_6 
[[0.05907673]
 [0.34862754]
 [0.6984816 ]]

rating: 4, P_7 
[[0.27337659]
 [0.29751156]
 [0.94986196]]



To teraz mnożymy rating przez P_i, dodajemy i wychodzi nam:

In [44]:
V_u = np.zeros(d).T
for i in I_u:
    V_u += ratings_matrix[u][i] * P[:, i]

V_u

array([4.27278025, 6.87352709, 8.6147099 ])

Jak już mamy wszystko policzone, możemy podstawić A_u oraz V_u do gaussa:

In [45]:
solution = gauss_method(A_u, V_u) # Tu jest gauss z zadania 2 z częściowym wyborem
solution

array([-0.45457671,  3.05289459,  3.9132431 ])

Jak już to policzyliśmy to zamieniamy kolumne u=0 w macierzy _U_ z naszym rozwiązaniem  
Macierz _U_ przed zamianą

In [46]:
U

array([[0.02930222, 0.90635812, 0.71271017],
       [0.03319273, 0.2316068 , 0.96492267],
       [0.35638381, 0.42064508, 0.83929454]])

Macierz _U_ po zamianie

In [47]:
U[:, u] = solution
U

array([[-0.45457671,  0.90635812,  0.71271017],
       [ 3.05289459,  0.2316068 ,  0.96492267],
       [ 3.9132431 ,  0.42064508,  0.83929454]])

Dla kolejnych _u_ postępujemy analogicznie

#### 5. for p = 1, 2, . . . m  
m = ilość produktów  
Tutaj zrobimy dla p = 0, (dla kolejnych _p_ analogicznie)

In [48]:
p = 0

Potrzebujemy _I\_p_, czyli indeksy użytkowników którzy ocenili dany produkt (ocena różna od 0) dla produktu _p_  
Bierzemy kolumnę o indeksie 0 i sprawdzamy gdzie ocena jest rózna od 0

In [49]:
I_p = np.flatnonzero(ratings_matrix[:, p])
I_p

array([1, 2])

Liczymy _U\_I\_p_ (czyli kolumny z macierzy _U_ o indeksach w _I\_p_)

In [50]:
U_I_p = U[:, I_p]
U_I_p

array([[0.90635812, 0.71271017],
       [0.2316068 , 0.96492267],
       [0.42064508, 0.83929454]])

Przy okazji możemy policzyć transponowane _U\_I\_p_

In [51]:
U_I_p_T = U_I_p.T
U_I_p_T

array([[0.90635812, 0.2316068 , 0.42064508],
       [0.71271017, 0.96492267, 0.83929454]])

Znowu liczymy macierz _E_ która jest macierzą jednostkową 3×3 (d×d)

In [52]:
E = np.eye(3)
E

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

Teraz możemy policzyć B_u

In [53]:
B_u = np.matmul(U_I_p ,U_I_p_T) + reg*E
B_u

array([[1.42944083, 0.89762891, 0.97942884],
       [0.89762891, 1.08471747, 0.9072786 ],
       [0.97942884, 0.9072786 , 0.98135762]])

Pozostało nam do policzenia _W\_p_ czyli musimy dodać do siebie:
 * kolumnę 1 z macierzy _U_ pomnożoną przez 4 (rating)
 * kolumnę 2 z macierzy _U_ pomnożoną przez 5 (rating)
 
 
 To wygląda mniej więcej tak:

In [54]:
for i in I_p:
    print('rating: {}, U_{} '.format(ratings_matrix[i][p], i))
    print(U[:, i].reshape(3,1))
    print()

rating: 4, U_1 
[[0.90635812]
 [0.2316068 ]
 [0.42064508]]

rating: 5, U_2 
[[0.71271017]
 [0.96492267]
 [0.83929454]]



To teraz mnożymy, dodajemy i wychodzi nam:

In [55]:
W_p = np.zeros(d)
for i in I_p:
    W_p += ratings_matrix[i, p] * U[:, i]

W_p

array([7.18898332, 5.75104058, 5.87905305])

Jak już mamy wszystko policzone, możemy podstawić _B\_p_ oraz _W\_u_ do gaussa:

In [56]:
solution = gauss_method(A_u, V_u) # Tu jest gauss z zadania 2 z częściowym wyborem
solution

array([-0.45457671,  3.05289459,  3.9132431 ])

Jak już to policzyliśmy to zamieniamy kolumne p=0 w macierzy _P_ z naszym rozwiązaniem  
Macierz _P_ przed zamianą

In [57]:
P

array([[0.93119636, 0.01215318, 0.82254304, 0.92704314, 0.72097256,
        0.1119594 , 0.05907673, 0.27337659, 0.51578453, 0.47299487],
       [0.1671686 , 0.02328032, 0.64793332, 0.46310597, 0.98508579,
        0.23390272, 0.34862754, 0.29751156, 0.81994987, 0.32293732],
       [0.72302848, 0.91165485, 0.70980305, 0.20125138, 0.33071352,
        0.40941998, 0.6984816 , 0.94986196, 0.52719633, 0.66722182]])

Po zamianie kolumny

In [58]:
P[:, p] = solution
P

array([[-0.45457671,  0.01215318,  0.82254304,  0.92704314,  0.72097256,
         0.1119594 ,  0.05907673,  0.27337659,  0.51578453,  0.47299487],
       [ 3.05289459,  0.02328032,  0.64793332,  0.46310597,  0.98508579,
         0.23390272,  0.34862754,  0.29751156,  0.81994987,  0.32293732],
       [ 3.9132431 ,  0.91165485,  0.70980305,  0.20125138,  0.33071352,
         0.40941998,  0.6984816 ,  0.94986196,  0.52719633,  0.66722182]])

Dla kolejnych _p_ postępujemy analogicznie

Koniec